In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os

source_path = '/content/dataset1_employees_initial_corrected.csv'
destination_dir = '/content/drive/MyDrive/Data Engineering Project/Datasets'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Construct the full destination path
destination_path = os.path.join(destination_dir, os.path.basename(source_path))

# Copy the file
shutil.copy(source_path, destination_path)

print(f"File copied successfully from '{source_path}' to '{destination_path}'")

In [3]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00


In [4]:
from faker import Faker
import random
import pandas as pd
from datetime import datetime, timedelta


In [5]:
import pandas as pd
import random
import re
from faker import Faker
from datetime import datetime, timedelta

# --- Configuration ---
fake = Faker()
MIN_HIRE_DATE = datetime(2025, 1, 1).date() # Only applies to hire_date

# List definitions
POSITIONS = [
    "CG", "CNA", "CHHA", "LPN", "RN", "ADMIN",
]

RAW_STATUSES = [
    "Active",
    "Active - Not Currently Available",
    "Active - Expired Credentials",
    "Resigned/Separated - Eligible for Rehire",
    "Resigned/Separated - Not Eligible for Rehire",
    "Terminated - Unemployable"
]

VALID_RATINGS = [
    "0 - Not Applicable",
    "4 - Exceeds Expectations",
    "3 - Meets Expectations",
    "1 - Unacceptable"
]

EDUCATION = ["High School", "Associate Degree", "4 year College"]


# --- Cleaning Functions ---

def clean_phone(phone):
    """Clean phone number to XXX-XXX-XXXX format."""
    if not phone:
        return None
    digits = re.sub(r"\D", "", str(phone))
    if len(digits) == 11 and digits.startswith("1"):
        digits = digits[1:]
    if len(digits) != 10:
        return None
    return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"

def clean_hire_date(hire_date, min_date=MIN_HIRE_DATE):
    """Ensure hire_date is not before MIN_HIRE_DATE (01/01/2025)."""
    if pd.isna(hire_date) or hire_date is None:
        return None

    # Ensure date is a date object
    if isinstance(hire_date, str):
        try:
            hire_date = pd.to_datetime(hire_date).date()
        except:
            return None

    # Check if hire_date is before the minimum required date
    if hire_date < min_date:
        # Shift forward by a random number of days from the minimum date, up to 1 year
        return min_date + timedelta(days=random.randint(0, 365))
    return hire_date

def clean_pay(rate):
    """Enforce pay rate between $15 and $75."""
    try:
        rate = float(rate)
        return min(max(rate, 15), 75)
    except:
        return None

# --- Main Generation Function ---

def generate_employee(employee_id_counter):
    """Generates a single, realistic, and clean employee record."""

    # 1. Generate core dates/age
    age = random.randint(19, 86)
    # Generate DOB realistically (can be historical)
    dob = fake.date_of_birth(minimum_age=age, maximum_age=age)

    # Generate a potential hire date (can be historical for the purpose of simulating old data,
    # but it will be cleaned/clamped later)
    potential_hire_date = fake.date_between(start_date='-18y', end_date='today')

    # 2. Generate dependent dates (based on historical date generation)
    # Ensure dependent dates are after the potential hire date (or near it)
    last_shift_date = fake.date_between(potential_hire_date, 'today')
    last_appraisal_date = fake.date_between(potential_hire_date, 'today')
    next_appraisal_date = fake.date_between('today', '+1y')
    cpr_date = fake.date_between('-2y', 'today')

    # 3. Compile raw data dictionary
    employee_data = {
        "employee_id": employee_id_counter,
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "address": fake.street_address(),
        "city": fake.city(),
        "state": fake.state_abbr(),
        "zip": fake.zipcode(),
        "primary_phone": fake.phone_number(),
        "email": fake.email(),
        "status": random.choice(RAW_STATUSES),
        "last_shift_date": last_shift_date,
        "position": random.choice(POSITIONS),
        "base_pay_rate": random.uniform(15, 75),
        "current_rating": random.choice(VALID_RATINGS),
        "last_appraisal_date": last_appraisal_date,
        "next_appraisal_date": next_appraisal_date,
        "hire_date": potential_hire_date,
        "terminated_date": None,
        "rehire_date": None,
        "cpr_date": cpr_date,
        "pto_balance": random.randint(0, 45),
        "commitment_hours": random.randint(0, 40),
        "age": age,
        "dob": dob,
        "gender": random.choice(["Male", "Female"]),
        "education": random.choice(EDUCATION),
        "languages": random.choice(["English", "English, Spanish", "English, French"]),
        "emergency_contact": fake.name(),
        "emergency_contact_phone": fake.phone_number(),
        "emergency_instructions": fake.sentence()
    }

    # 4. Apply cleaning/validation
    employee_data["primary_phone"] = clean_phone(employee_data["primary_phone"])
    employee_data["emergency_contact_phone"] = clean_phone(employee_data["emergency_contact_phone"])
    employee_data["base_pay_rate"] = clean_pay(employee_data["base_pay_rate"])

    # Apply date cleaning ONLY to the hire_date
    employee_data["hire_date"] = clean_hire_date(employee_data["hire_date"])

    return employee_data

# --- Execution ---

DATASET_SIZE = 30
data = []
for i in range(1, DATASET_SIZE + 1):
    data.append(generate_employee(i))

df = pd.DataFrame(data)

# Save the final cleaned dataset
OUTPUT_FILENAME = "dataset1_employees_initial_corrected.csv"
df.to_csv(OUTPUT_FILENAME, index=False)

print(f"Successfully generated and cleaned {DATASET_SIZE} rows.")
print(f"Saved to: {OUTPUT_FILENAME}")
print("\nFirst 5 rows of the generated dataset (Note: Historical dates are now allowed for non-hire columns):")
print(df.head(30).to_markdown(index=False))

Successfully generated and cleaned 30 rows.
Saved to: dataset1_employees_initial_corrected.csv

First 5 rows of the generated dataset (Note: Historical dates are now allowed for non-hire columns):
|   employee_id | first_name   | last_name   | address                            | city           | state   |   zip | primary_phone   | email                        | status                                       | last_shift_date   | position   |   base_pay_rate | current_rating           | last_appraisal_date   | next_appraisal_date   | hire_date   | terminated_date   | rehire_date   | cpr_date   |   pto_balance |   commitment_hours |   age | dob        | gender   | education        | languages        | emergency_contact   | emergency_contact_phone   | emergency_instructions                                   |
|--------------:|:-------------|:------------|:-----------------------------------|:---------------|:--------|------:|:----------------|:-----------------------------|:---------------